In [1]:
import pandas as pd
# 读取数据
data = pd.read_csv("../machine_learn/data/FBlocation/train.csv")

# 处理数据
# 1、缩小数据,查询数据,为了减少计算时间
data = data.query("x > 1.0 &  x < 1.25 & y > 2.5 & y < 2.75")

#2. 处理时间的数据
time_value = pd.to_datetime(data['time'], unit='s')
time_value = pd.DatetimeIndex(time_value)

#3  构造一些特征，执行的警告是因为我们的操作是复制，loc是直接放入
data.insert(data.shape[1], 'day', time_value.day)
data.insert(data.shape[1], 'hour', time_value.hour)
data.insert(data.shape[1], 'weekday', time_value.weekday)

data = data.drop(['time'], axis=1)#axis=1代表列

place_count = data.groupby('place_id').count() #对place_id进行分组，然后统计每个place_id的个数

tf = place_count[place_count.row_id > 3].reset_index()

data = data[data['place_id'].isin(tf.place_id)]

#4、取出数据中的特征值和目标值
y = data['place_id'] #目标值
x = data.drop(['place_id'], axis=1)
x = x.drop(['row_id'], axis=1)

from sklearn.model_selection import train_test_split # 数据的分割
from sklearn.preprocessing import StandardScaler # 数据的标准化
from sklearn.neighbors import KNeighborsClassifier


# 进行数据的分割训练集合测试集
x_train, x_test, y_train, y_test = train_test_split(
    x,
    y,
    test_size=0.25,
    random_state=1)
std = StandardScaler()
x_train = std.fit_transform(x_train)
x_test = std.transform(x_test)


# # 进行算法流程 # 超参数，可以通过设置n_neighbors=5，来调整结果好坏
knn = KNeighborsClassifier(n_neighbors=6)

In [2]:
from sklearn.model_selection import GridSearchCV

#网格搜索时讲解
# # 构造一些参数的值进行搜索
# dict---<string,list(int)/list(string)>
param = {"n_neighbors": [3, 5, 10, 12, 15],'weights':['uniform', 'distance']}
# print(type(param))
# print(type(param["n_neighbors"]))


# 进行网格搜索，cv=3是3折交叉验证，用其中2折训练，1折验证
gc = GridSearchCV(knn, param_grid=param, cv=3)

gc.fit(x_train, y_train)  #你给它的x_train，它又分为训练集，验证集

# 预测准确率，为了给大家看看
print("在测试集上准确率：", gc.score(x_test, y_test))

print("在交叉验证当中最好的结果：", gc.best_score_)

print("选择最好的模型是：", gc.best_estimator_)

print("每个超参数每次交叉验证的结果：", gc.cv_results_)
gc.cv_results_

D:\PY\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


在测试集上准确率： 0.49763593380614657
在交叉验证当中最好的结果： 0.4816362349278435
选择最好的模型是： KNeighborsClassifier(n_neighbors=12, weights='distance')
每个超参数每次交叉验证的结果： {'mean_fit_time': array([0.01129659, 0.01262403, 0.01162553, 0.01195884, 0.0132885 ,
       0.01183216, 0.01162664, 0.01162807, 0.01262673, 0.01162672]), 'std_fit_time': array([4.66256546e-04, 4.70527427e-04, 4.69186078e-04, 2.24502037e-06,
       1.87907510e-03, 6.64593584e-04, 4.71314248e-04, 4.70302724e-04,
       4.72100989e-04, 4.68675543e-04]), 'mean_score_time': array([0.15203301, 0.05348659, 0.1530052 , 0.06611196, 0.17150768,
       0.08837128, 0.17142924, 0.09686573, 0.18228316, 0.11616802]), 'std_score_time': array([0.00639963, 0.00204684, 0.0018937 , 0.00093982, 0.00129869,
       0.00235095, 0.00215281, 0.00288094, 0.00441415, 0.00557425]), 'param_n_neighbors': masked_array(data=[3, 3, 5, 5, 10, 10, 12, 12, 15, 15],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
      

{'mean_fit_time': array([0.01129659, 0.01262403, 0.01162553, 0.01195884, 0.0132885 ,
        0.01183216, 0.01162664, 0.01162807, 0.01262673, 0.01162672]),
 'std_fit_time': array([4.66256546e-04, 4.70527427e-04, 4.69186078e-04, 2.24502037e-06,
        1.87907510e-03, 6.64593584e-04, 4.71314248e-04, 4.70302724e-04,
        4.72100989e-04, 4.68675543e-04]),
 'mean_score_time': array([0.15203301, 0.05348659, 0.1530052 , 0.06611196, 0.17150768,
        0.08837128, 0.17142924, 0.09686573, 0.18228316, 0.11616802]),
 'std_score_time': array([0.00639963, 0.00204684, 0.0018937 , 0.00093982, 0.00129869,
        0.00235095, 0.00215281, 0.00288094, 0.00441415, 0.00557425]),
 'param_n_neighbors': masked_array(data=[3, 3, 5, 5, 10, 10, 12, 12, 15, 15],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value='?',
             dtype=object),
 'param_weights': masked_array(data=['uniform', 'distance', 'uniform', 'distance',
      